# Model Connectivity Overview
## Setup
### Import packages

In [ ]:
from collections import defaultdict
from itertools import permutations

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from cobra.util.array import create_stoichiometric_matrix
from rbc_gem_utils import (
    GEM_NAME,
    build_string,
    get_dirpath,
    read_cobra_model,
    show_versions,
)
from rbc_gem_utils.util import ensure_iterable

# Display versions of last time notebook ran and worked
show_versions()
plt.rcParams["font.family"] = "Arial"

## Load RBC-GEM model

In [ ]:
model_dirpath = get_dirpath("model")
model = read_cobra_model(filename=model_dirpath / f"{GEM_NAME}.xml")


model.metabolites.sort()
model.genes.sort()
model.reactions.sort()
model

In [ ]:
save_figures = True
transparent = False
imagetype = "svg"
dpi = 600
include_boundary_in_calculations = True
results_path = get_dirpath(use_temp="processed") / model.id / "ConnectivityOverview"
results_path.mkdir(exist_ok=True, parents=True)

### Matrix Sparsity

In [ ]:
model_sparsity = model.copy()

# Reorder metabolites, first intracellular, then extracellular
cytosol_metabolites = model_sparsity.metabolites.query(lambda x: x.compartment == "c")
extracellular_metabolites = model_sparsity.metabolites.query(
    lambda x: x.compartment == "e"
)
metabolites_ordered = cytosol_metabolites + extracellular_metabolites
assert len(metabolites_ordered) == len(model_sparsity.metabolites)
model_sparsity.metabolites = metabolites_ordered

# Reorder reactions, first intracellular, then transport, finally pseudoreactions
intracellular_reactions = model_sparsity.reactions.query(
    lambda x: not x.boundary and len(model_sparsity.compartments) == 1
)
transport_reactions = model_sparsity.reactions.query(
    lambda x: not x.boundary and len(model_sparsity.compartments) > 1
)
boundary_reactions = model_sparsity.reactions.query(lambda x: x.boundary)
reactions_ordered = intracellular_reactions + transport_reactions + boundary_reactions
assert len(reactions_ordered) == len(model_sparsity.reactions)
model_sparsity.reactions = reactions_ordered
model_sparsity.repair()

In [ ]:
# Ratio of reactions to metabolites currently ~1.4 to 1
print(
    f"Reactions:metabolites ({len(model_sparsity.reactions) / len(model_sparsity.metabolites):.2f} : {len(model_sparsity.metabolites) / len(model_sparsity.metabolites):.2f})"
)

fig, ax = plt.subplots(1, 1, figsize=(1.4 * 4 + 1, 4 + 1))
S = create_stoichiometric_matrix(model, array_type="DataFrame")
if not include_boundary_in_calculations:
    S = S.loc[
        :, model_sparsity.reactions.query(lambda x: not x.boundary).list_attr("id")
    ]
sparsity = 1.0 - (np.count_nonzero(S) / float(S.size))

ax.spy(S, markersize=1, color="black")
ax.set_title(
    f"{model_sparsity.id.replace('_', '-')}: Stoichiometric Matrix\n(Sparsity = {sparsity*100:.2f}%)",
    fontsize=12,
)
ax.set_xlabel(f"Reactions (n={len(model_sparsity.reactions)})", fontsize=12)
ax.set_ylabel(f"Metabolites (n={len(model_sparsity.metabolites)})", fontsize=12)
ax.xaxis.set_ticks_position("bottom")
metabolite_indicies = dict(
    zip(
        list(range(1 + len(model_sparsity.metabolites.list_attr("id")) + 2)),
        model_sparsity.metabolites.list_attr("id"),
    )
)
metabolite_indicies = pd.DataFrame.from_dict(
    dict(
        zip(
            list(range(1 + len(model_sparsity.metabolites.list_attr("id")) + 2)),
            model_sparsity.metabolites.list_attr("id"),
        )
    ),
    orient="index",
    columns=["metabolites"],
)


reaction_indicies = dict(
    zip(
        list(range(1 + len(model_sparsity.reactions.list_attr("id")) + 2)),
        model_sparsity.reactions.list_attr("id"),
    )
)
reaction_indicies = pd.DataFrame.from_dict(
    dict(
        zip(
            list(range(1 + len(model_sparsity.reactions.list_attr("id")) + 2)),
            model_sparsity.reactions.list_attr("id"),
        )
    ),
    orient="index",
    columns=["reactions"],
)
fig.tight_layout()
if save_figures:
    fig.savefig(
        results_path / f"MatrixSparsity_{model.id}.{imagetype}",
        transparent=transparent,
    )

### Calculate connectivities

In [ ]:
table_data = {}

In [ ]:
def compute_connectivities(
    model,
    attribute_type="metabolites",
    compartments=None,
    exclude=None,
    sum_compartments=False,
    aggregate_ranking=False,
    include_boundary=False,
):
    """Compute the metabolite connectivities of the model.

    Parameters
    ----------
    model : Model
        The model object
    attribute_type : {"metabolites", "genes"}
        The type of object.
    compartments : list, None
        Compartments that should be accounted for. If ``None``, all metabolite compartments are utilized.
    exclude : iterable
        Contains identifiers for metabolites that should be excluded from calculations.
        Useful for excluding currency metabolites (e.g., hydrogen, water) from calculations.
    sum_compartments : bool
        Whether to treat the metabolite as one entity and sum metabolite connectivities across compartments.
        If True, the connectivity is determined without regard to the species compartment.
        If False, the connectivity is determined seperately for species in different compartments.
    aggregate_ranking : bool
        If True, metabolites that have the same connectivity value are grouped together.
        If False, metabolites are kept seperately.
    include_boundary : bool
        If True, pseudoreactions are included in the connectivity count.
        If False, metabolites are kept seperately.
    """
    if not include_boundary:
        # Copy model and remove boundaries
        model = model.copy()
        model.remove_groups([model.groups.get_by_id("Pseudoreactions")])
        model.remove_reactions(model.boundary)
    species = getattr(model, attribute_type)
    # Get compartments
    if compartments:
        compartments = ensure_iterable(compartments)
    else:
        compartments = sorted(model.compartments)

    # Exclude metabolites
    if exclude is None:
        exclude = set()
    else:
        exclude = {str(x) for x in ensure_iterable(exclude)}

    if attribute_type == "metabolites":
        species = species.query(
            lambda x: (
                (
                    not x.id.replace(f"_{x.compartment}", "") in exclude
                    and not x.id in exclude
                )
                if sum_compartments
                else (not x.id in exclude)
            )
        )
        species = species.query(lambda x: x.compartment in set(compartments))
    else:
        species = species.query(lambda x: not x.id in exclude)

    # Compute absolute connectivities
    absolute_connectivities = defaultdict(int)
    absolute_connectivities = pd.Series(
        {specie.id: len(specie.reactions) for specie in species}, name="absolute"
    ).sort_values(ascending=False)
    absolute_connectivities.index.name = attribute_type

    if sum_compartments and attribute_type == "metabolites":
        # Aggregate metabolites across compartments and sum
        absolute_connectivities = absolute_connectivities.rename(
            {
                sid: sid.replace(f"_{species.get_by_id(sid).compartment}", "")
                for sid in absolute_connectivities.index
            },
            axis=0,
        )
        absolute_connectivities = absolute_connectivities.groupby(
            absolute_connectivities.index
        ).sum()

    # Compute relative connectivities
    relative_connectivities = absolute_connectivities / absolute_connectivities.sum()
    relative_connectivities.name = "relative"
    # Create DataFrame
    df_connectivity = pd.merge(
        absolute_connectivities,
        relative_connectivities,
        left_index=True,
        right_index=True,
    )
    df_connectivity = df_connectivity.reset_index(drop=False)
    df_connectivity = df_connectivity.sort_values(
        ["absolute", attribute_type], ascending=False
    )

    if aggregate_ranking:
        # Aggregate metabolites that share the same ranking
        df_connectivity = (
            df_connectivity.groupby(["absolute", "relative"], as_index=False)
            .agg(lambda x: set(x))
            .sort_index(ascending=False)
        )
    # Reset index, reorganize columns, and return
    df_connectivity = df_connectivity.reset_index(drop=True).loc[
        :, [attribute_type, "absolute", "relative"]
    ]
    return df_connectivity

#### Metabolite connectivity

In [ ]:
num_display = 15
sum_compartments = True
aggregate_ranking = False
include_boundary = False
exclude_list = []
attribute_type = "metabolites"

compartments = ["e"]
df_connectivity = compute_connectivities(
    model,
    attribute_type=attribute_type,
    compartments=compartments,
    exclude=exclude_list,
    sum_compartments=sum_compartments,
    aggregate_ranking=aggregate_ranking,
)
df_connectivity[attribute_type] = df_connectivity[attribute_type].apply(
    lambda x: build_string(x, sep=", ")
)
df_connectivity = df_connectivity.set_index("metabolites")
df_connectivity.columns = pd.MultiIndex.from_product(
    [[model.compartments["e"]], df_connectivity.columns]
)
df_connectivity_e = df_connectivity.copy()

compartments = ["c"]
df_connectivity = compute_connectivities(
    model,
    attribute_type=attribute_type,
    compartments=compartments,
    exclude=exclude_list,
    sum_compartments=sum_compartments,
    aggregate_ranking=aggregate_ranking,
)
df_connectivity[attribute_type] = df_connectivity[attribute_type].apply(
    lambda x: build_string(x, sep=", ")
)
df_connectivity = df_connectivity.set_index("metabolites")
df_connectivity.columns = pd.MultiIndex.from_product(
    [[model.compartments["c"]], df_connectivity.columns]
)
df_connectivity_c = df_connectivity.copy()

df_metabolite_connectivity_table = pd.merge(
    df_connectivity_c,
    df_connectivity_e,
    left_index=True,
    right_index=True,
    how="outer",
)

compartments = None
df_connectivity = compute_connectivities(
    model,
    attribute_type=attribute_type,
    compartments=compartments,
    exclude=exclude_list,
    sum_compartments=sum_compartments,
    aggregate_ranking=aggregate_ranking,
)
df_connectivity[attribute_type] = df_connectivity[attribute_type].apply(
    lambda x: build_string(x, sep=", ")
)
df_connectivity = df_connectivity.set_index("metabolites")
df_connectivity.columns = pd.MultiIndex.from_product(
    [["total"], df_connectivity.columns]
)
df_connectivity_all = df_connectivity.copy()

df_metabolite_connectivity_table = (
    pd.merge(
        df_connectivity_all,
        df_metabolite_connectivity_table,
        left_index=True,
        right_index=True,
        how="outer",
    )
    .fillna(0)
    .sort_values(by=("total", "absolute"), ascending=False)
)
for col in df_metabolite_connectivity_table.columns:
    if col[1] == "absolute":
        df_metabolite_connectivity_table[col] = df_metabolite_connectivity_table[
            col
        ].astype(int)
    else:
        df_metabolite_connectivity_table[col] = df_metabolite_connectivity_table[
            col
        ].astype(float)
table_data[attribute_type] = df_metabolite_connectivity_table.copy()
df_metabolite_connectivity_table.head(num_display)

In [ ]:
df_connectivity = df_metabolite_connectivity_table["total"].reset_index(drop=False)


key = "absolute"
scale_points = False
color = "xkcd:light red"
refline_color = "black"

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
sns.despine(fig)


ax.scatter(
    df_connectivity[key].index + 1,  # Log plots do not do well with index of 0.
    df_connectivity[key].values,
    marker="o",
    edgecolors="black",
    lw=0.5,
    # linestyle="",
    color=color,
    s=df_connectivity["relative"].values * 10000 if scale_points else 100,
    zorder=2,
    label=model.id.replace("_", "-"),
)
# A reference line, linking the minimum and maximum connectivities
ax.loglog(
    [df_connectivity[key].max(), df_connectivity[key].min()],
    [df_connectivity[key].min(), df_connectivity[key].max()],
    zorder=1,
    color=refline_color,
    linestyle=":",
)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_title("Metabolite Connectivity", fontdict={"size": 20}, y=1.1)
ax.set_xlabel("Rank", fontdict={"size": 16})
ax.set_ylabel("Number of associated reactions", fontdict={"size": 16})
ax.set_xlim(
    (0.9, df_connectivity[key].max() * 2)
)  # Add some padding for figure clarity
ax.set_ylim(
    (0.9, df_connectivity[key].max() * 2)
)  # Add some padding for figure clarity
ax.set_xticks([1e0, 1e1, 1e2, 1e3])
ax.set_yticks([1e0, 1e1, 1e2, 1e3])
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)


mets_to_labels = {
    # Name, xscalar, yscalar
    "h": ("H$^+$", 1, 2.5),
    "na1": ("Na$^+$", 0.75, 7),
    "atp": ("ATP", 1.25, 6),
    "amp": ("AMP", 2.5, 4),
    "nadp": ("NADP(H)", 4.5, 10),
    "nad": ("NAD(H)", 9, 9),
    "coa": ("CoA", 1.8, 3),
    "o2": ("O$_2$", 12, 4),
    "ala__L": ("ALA", 25, 3.2),
    "ser__L": ("SER", 23, 1.6),
    "cys__L": ("CYS", 22, 0.8),
    "gln__L": ("GLN", 21.5, 0.4),
    "gly": ("GLY", 20, 0.2),
}
for i, row in df_connectivity[
    df_connectivity["metabolites"].isin(list(mets_to_labels))
].iterrows():
    x = i + 1
    met, y = row[["metabolites", "absolute"]].values
    name, xscalar, yscalar = mets_to_labels[met]
    ax.annotate(
        xy=(x, y),
        xytext=(x * xscalar, y * yscalar),
        text=name,
        xycoords="data",
        fontsize=14,
        arrowprops=dict(arrowstyle="->"),
    )

fig.tight_layout()
if save_figures:
    fig.savefig(
        results_path / f"{attribute_type}_connectivity_{model.id}.{imagetype}",
        transparent=transparent,
        dpi=None if imagetype != "png" else 600,
    )

### Gene connectivity

In [ ]:
num_display = 15
compartments = None
sum_compartments = True
aggregate_ranking = False
include_boundary = False
exclude_list = []
attribute_type = "genes"


df_connectivity = compute_connectivities(
    model,
    attribute_type=attribute_type,
    compartments=compartments,
    exclude=exclude_list,
    sum_compartments=sum_compartments,
    aggregate_ranking=aggregate_ranking,
)
df_connectivity[attribute_type] = df_connectivity[attribute_type].apply(
    lambda x: build_string(x, sep=", ")
)
table_data[attribute_type] = df_connectivity.copy()
df_connectivity.iloc[num_display : num_display * 2].head(num_display)

In [ ]:
key = "absolute"
scale_points = False
color = "xkcd:light red"
refline_color = "black"

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
sns.despine(fig)


ax.scatter(
    df_connectivity[key].index + 1,  # Log plots do not do well with index of 0.
    df_connectivity[key].values,
    marker="o",
    edgecolors="black",
    lw=0.5,
    # linestyle="",
    color=color,
    s=df_connectivity["relative"].values * 10000 if scale_points else 100,
    zorder=2,
    label=model.id.replace("_", "-"),
)
# A reference line, linking the minimum and maximum connectivities
ax.loglog(
    [df_connectivity[key].max(), df_connectivity[key].min()],
    [df_connectivity[key].min(), df_connectivity[key].max()],
    zorder=1,
    color=refline_color,
    linestyle=":",
)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_title("Gene Connectivity", fontdict={"size": 20}, y=1.1)
ax.set_xlabel("Rank", fontdict={"size": 16})
ax.set_ylabel("Number of associated reactions", fontdict={"size": 16})
ax.set_xlim(
    (0.9, df_connectivity[key].max() * 2)
)  # Add some padding for figure clarity
ax.set_ylim(
    (0.9, df_connectivity[key].max() * 2)
)  # Add some padding for figure clarity
ax.set_xticks([1e0, 1e1, 1e2, 1e3])
ax.set_yticks([1e0, 1e1, 1e2, 1e3])
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)

fig.tight_layout()
if save_figures:
    fig.savefig(
        results_path / f"{attribute_type}_connectivity_{model.id}.{imagetype}",
        transparent=transparent,
    )

In [ ]:
print(list(table_data.keys()))
if save_figures:
    with pd.ExcelWriter(
        results_path / f"ModelConnectivity_{model.id}.xlsx",
    ) as writer:
        for sheet_name, df in table_data.items():
            df.to_excel(writer, sheet_name=sheet_name, index=True)

#### Calculate connectivities with exclusion of currency/common hub metabolites

In [ ]:
def get_primary_reactions(model, metabolite_set):
    """Return the primary reactions for a set of metabolites.

    Used to help determine reactions which reactions treat cofactors as a primary or secondary metabolite product.
    """
    primary_reactions = set()
    for combo in permutations(list(metabolite_set), 2):
        main_met = model.metabolites.get_by_id(combo[0])
        for other_met in combo[1:]:
            other_met = model.metabolites.get_by_id(other_met)
            for reaction in main_met.reactions:
                if not reaction in other_met.reactions:
                    primary_reactions.add(reaction.id)
    return primary_reactions


def remove_metabolites_from_reactions(model, metabolite_set, exclusion_list):
    """Helper method to remove metabolites from reactions."""
    new_model = model.copy()
    for metabolite in list(metabolite_set):
        metabolite = new_model.metabolites.get_by_id(metabolite)
        for reaction in metabolite.reactions:
            reaction = new_model.reactions.get_by_id(reaction.id)
            if reaction.id in exclusion_list:
                continue
            reaction.add_metabolites({metabolite: 0}, combine=False)

    new_model.repair()
    return new_model

In [ ]:
mets_to_exclude = [
    "h",
    "oh1",
    "h2o",
    "k",
    "na1",
    "cl",
    "ag1",
    "ca2",
    "cd2",
    "cobalt2",
    "cu1",
    "cu2",
    "fe2",
    "fe3",
    "mg2",
    "mn2",
    "zn2",
    "i",
    "o2",
    "o2s",
    "co2",
    "h2o2",
    "nh4",
    "nh3",
    "pi",
    "ppi",
    "so4",
    "so3",
    "coa",
    "hco3",
]
model_excluded_mets = model.copy()
for compartment in ["c", "e"]:
    for metabolite_set in [
        # Common Redox pairs
        # NADH <=> NAD
        # NADPH <=> NADP
        # 2 GTHRD <=> GTHOX
        ["nadh", "nad"],
        ["nadph", "nadp"],
        ["gthrd", "gthox"],
    ]:
        metabolite_set = [f"{met}_{compartment}" for met in metabolite_set]
        try:
            exclude_from_changes = get_primary_reactions(
                model_excluded_mets, metabolite_set
            )
            model_excluded_mets = remove_metabolites_from_reactions(
                model_excluded_mets, metabolite_set, exclude_from_changes
            )
        except KeyError:
            print(f"Not all metabolites present for {metabolite_set}")

for compartment in ["c", "e"]:
    for metabolite_set in [
        # Coupled to hydrolysis of ATP
        # ATP --> AMP + PPi
        # ATP --> ADP + Pi
        # ATP --> ADP
        ["atp", "amp", "ppi"],
        ["atp", "adp", "pi"],
        ["atp", "adp"],
    ]:
        metabolite_set = [f"{met}_{compartment}" for met in metabolite_set]
        try:
            exclude_from_changes = get_primary_reactions(
                model_excluded_mets, metabolite_set
            )

            exclude_from_changes.update(
                [
                    "ADK1",  # Needs manual correction
                    "AP5AH1",  # Should not be corrected
                ]
            )
            model_excluded_mets = remove_metabolites_from_reactions(
                model_excluded_mets, metabolite_set, exclude_from_changes
            )
        except KeyError:
            print(f"Not all metabolites present for {metabolite_set}")
# Remove secondary ATP --> ADP coupled to transformation
model_excluded_mets.reactions.ADK1.build_reaction_from_string("amp_c <=> adp_c")
for compartment in ["c", "e"]:
    model_excluded_mets.remove_metabolites(
        [
            model_excluded_mets.metabolites.get_by_id(f"{met}_{compartment}")
            for met in mets_to_exclude
            if model_excluded_mets.metabolites.has_id(f"{met}_{compartment}")
        ],
        destructive=False,
    )

model_excluded_mets.remove_reactions(
    model_excluded_mets.reactions.query(lambda x: len(x.metabolites) == 0)
)
model_excluded_mets.remove_metabolites(
    model_excluded_mets.metabolites.query(lambda x: len(x.reactions) == 0)
)
model_excluded_mets.metabolites.sort()
model_excluded_mets.genes.sort()
model_excluded_mets.reactions.sort()
model_excluded_mets.groups.sort()
model_excluded_mets.repair()
model_excluded_mets

In [ ]:
for met in sorted(mets_to_exclude):
    print(model.metabolites.get_by_id(f"{met}_c").name)

In [ ]:
num_display = 15
sum_compartments = True
aggregate_ranking = False
include_boundary = False
attribute_type = "metabolites"
exclude_list = []

model = model_excluded_mets.copy()
compartments = ["e"]
df_connectivity = compute_connectivities(
    model,
    attribute_type=attribute_type,
    compartments=compartments,
    exclude=exclude_list,
    sum_compartments=sum_compartments,
    aggregate_ranking=aggregate_ranking,
)
df_connectivity[attribute_type] = df_connectivity[attribute_type].apply(
    lambda x: build_string(x, sep=", ")
)
df_connectivity = df_connectivity.set_index("metabolites")
df_connectivity.columns = pd.MultiIndex.from_product(
    [[model.compartments["e"]], df_connectivity.columns]
)
df_connectivity_e = df_connectivity.copy()

compartments = ["c"]
df_connectivity = compute_connectivities(
    model,
    attribute_type=attribute_type,
    compartments=compartments,
    exclude=exclude_list,
    sum_compartments=sum_compartments,
    aggregate_ranking=aggregate_ranking,
)
df_connectivity[attribute_type] = df_connectivity[attribute_type].apply(
    lambda x: build_string(x, sep=", ")
)
df_connectivity = df_connectivity.set_index("metabolites")
df_connectivity.columns = pd.MultiIndex.from_product(
    [[model.compartments["c"]], df_connectivity.columns]
)
df_connectivity_c = df_connectivity.copy()

df_metabolite_connectivity_table = pd.merge(
    df_connectivity_c,
    df_connectivity_e,
    left_index=True,
    right_index=True,
    how="outer",
)

compartments = None
df_connectivity = compute_connectivities(
    model,
    attribute_type=attribute_type,
    compartments=compartments,
    exclude=exclude_list,
    sum_compartments=sum_compartments,
    aggregate_ranking=aggregate_ranking,
)
df_connectivity[attribute_type] = df_connectivity[attribute_type].apply(
    lambda x: build_string(x, sep=", ")
)
df_connectivity = df_connectivity.set_index("metabolites")
df_connectivity.columns = pd.MultiIndex.from_product(
    [["total"], df_connectivity.columns]
)
df_connectivity_all = df_connectivity.copy()

df_metabolite_connectivity_table = (
    pd.merge(
        df_connectivity_all,
        df_metabolite_connectivity_table,
        left_index=True,
        right_index=True,
        how="outer",
    )
    .fillna(0)
    .sort_values(by=("total", "absolute"), ascending=False)
)
for col in df_metabolite_connectivity_table.columns:
    if col[1] == "absolute":
        df_metabolite_connectivity_table[col] = df_metabolite_connectivity_table[
            col
        ].astype(int)
    else:
        df_metabolite_connectivity_table[col] = df_metabolite_connectivity_table[
            col
        ].astype(float)

table_data[f"{attribute_type}-con-exclude"] = df_metabolite_connectivity_table.copy()
df_metabolite_connectivity_table.head(num_display)

In [ ]:
table_data["MetConnectivityExclusions"] = df_metabolite_connectivity_table.copy()
table_data["MetConnectivityExclusions"]

##### Visualize

In [ ]:
df = pd.concat(
    (
        df_metabolite_connectivity_table["cytosol"]["absolute"],
        df_metabolite_connectivity_table["extracellular space"]["absolute"],
    ),
    axis=1,
).head(10)
df.columns = ["cytosol", "extracellular space"]


columns_to_plot = ["cytosol", "extracellular space"]
df = df[columns_to_plot].copy()
colormaps_dict = {
    columns_to_plot[0]: mpl.colormaps["Paired"](1),
    columns_to_plot[1]: mpl.colormaps["Paired"](0),
}

mets_map = {
    "crn": "L-carnitine",
    "ala__L": "L-alanine",
    "ser__L": "L-serine",
    "cys__L": "L-cysteine",
    "gln__L": "L-glutamine",
    "thr__L": "L-threonine",
    "gly": "Glycine",
    "asn__L": "L-asparagine",
    "pro__L": "L-proline",
    "leu__L": "L-leucine",
}
df.index = [mets_map[met] for met in df.index]


fig, ax = plt.subplots(1, 1, figsize=(4, 3))
sns.despine(fig)

barsize = 0.75
edgecolor = "black"
edgewidth = 1
padding = 2
offset = np.zeros(len(df.index))
df.index = df.index.astype(str)
df = df.loc[df.index[::-1]]
for idx, (label, series) in enumerate(df.items()):
    rects = ax.barh(
        series.index,
        series.values,
        height=barsize,
        left=offset,
        color=colormaps_dict.get(label),
        edgecolor=edgecolor,
        linewidth=edgewidth,
    )
    offset += series.values

ax.xaxis.set_tick_params(labelsize="large")
ax.yaxis.set_tick_params(labelsize="large")

ax.set_xlabel("Number of associated reactions", fontdict={"size": "large"})
ax.set_xticks([0, 30, 60, 90, 120])

ax.legend(
    handles=[
        mpl.patches.Patch(facecolor=color, edgecolor=edgecolor)
        for color in list(colormaps_dict.values())
    ],
    labels=list(colormaps_dict.keys()),
    frameon=False,
    bbox_to_anchor=(0.5, 1),
    loc="lower center",
    ncol=2,
    fontsize="large",
)
fig.tight_layout()
if save_figures:
    fig.savefig(
        results_path / f"Top10_w_transporter.{imagetype}",
        transparent=transparent,
        dpi=None if imagetype != "png" else dpi,
    )

#### Calculate connectivities for intracellular network only

In [ ]:
model_excluded_transports = model_excluded_mets.copy()
model_excluded_transports.remove_reactions(
    [
        reaction
        for reaction in model_excluded_transports.reactions
        if reaction.compartments != {"c"}
    ]
)

In [ ]:
num_display = 15
sum_compartments = True
aggregate_ranking = False
include_boundary = False
attribute_type = "metabolites"
exclude_list = []

model = model_excluded_transports.copy()
compartments = ["e"]
df_connectivity = compute_connectivities(
    model,
    attribute_type=attribute_type,
    compartments=compartments,
    exclude=exclude_list,
    sum_compartments=sum_compartments,
    aggregate_ranking=aggregate_ranking,
)
df_connectivity[attribute_type] = df_connectivity[attribute_type].apply(
    lambda x: build_string(x, sep=", ")
)
# table_data[attribute_type] = df_connectivity
df_connectivity = df_connectivity.set_index("metabolites")
df_connectivity.columns = pd.MultiIndex.from_product(
    [[model.compartments["e"]], df_connectivity.columns]
)
df_connectivity_e = df_connectivity.copy()

compartments = ["c"]
df_connectivity = compute_connectivities(
    model,
    attribute_type=attribute_type,
    compartments=compartments,
    exclude=exclude_list,
    sum_compartments=sum_compartments,
    aggregate_ranking=aggregate_ranking,
)
df_connectivity[attribute_type] = df_connectivity[attribute_type].apply(
    lambda x: build_string(x, sep=", ")
)
# table_data[attribute_type] = df_connectivity
df_connectivity = df_connectivity.set_index("metabolites")
df_connectivity.columns = pd.MultiIndex.from_product(
    [[model.compartments["c"]], df_connectivity.columns]
)
df_connectivity_c = df_connectivity.copy()

df_metabolite_connectivity_table = pd.merge(
    df_connectivity_c,
    df_connectivity_e,
    left_index=True,
    right_index=True,
    how="outer",
)

compartments = None
df_connectivity = compute_connectivities(
    model,
    attribute_type=attribute_type,
    compartments=compartments,
    exclude=exclude_list,
    sum_compartments=sum_compartments,
    aggregate_ranking=aggregate_ranking,
)
df_connectivity[attribute_type] = df_connectivity[attribute_type].apply(
    lambda x: build_string(x, sep=", ")
)
# table_data[attribute_type] = df_connectivity
df_connectivity = df_connectivity.set_index("metabolites")
df_connectivity.columns = pd.MultiIndex.from_product(
    [["total"], df_connectivity.columns]
)
df_connectivity_all = df_connectivity.copy()

df_metabolite_connectivity_table = (
    pd.merge(
        df_connectivity_all,
        df_metabolite_connectivity_table,
        left_index=True,
        right_index=True,
        how="outer",
    )
    .fillna(0)
    .sort_values(by=("total", "absolute"), ascending=False)
)
for col in df_metabolite_connectivity_table.columns:
    if col[1] == "absolute":
        df_metabolite_connectivity_table[col] = df_metabolite_connectivity_table[
            col
        ].astype(int)
    else:
        df_metabolite_connectivity_table[col] = df_metabolite_connectivity_table[
            col
        ].astype(float)


table_data[f"{attribute_type}-con-exclude_no_tpt"] = (
    df_metabolite_connectivity_table.copy()
)
df_metabolite_connectivity_table.head(num_display)

In [ ]:
table_data["MetConnectivityNoTransport"] = df_metabolite_connectivity_table.copy()
table_data["MetConnectivityNoTransport"]

##### Visualize

In [ ]:
df = pd.concat(
    (
        df_metabolite_connectivity_table["cytosol"]["absolute"],
        df_metabolite_connectivity_table["extracellular space"]["absolute"],
    ),
    axis=1,
).head(10)
df.columns = ["cytosol", "extracellular space"]


columns_to_plot = ["cytosol", "extracellular space"]
df = df[columns_to_plot].copy()
colormaps_dict = {
    columns_to_plot[0]: mpl.colormaps["Paired"](8),
    columns_to_plot[1]: mpl.colormaps["Paired"](9),
}

mets_map = {
    "crn": "L-carnitine",
    "glu__L": "L-glutamate",
    "avite1qn": "Alpha-tocopherol quinone",
    "amp": "AMP",
    "udp": "UDP",
    "avite1": "Alpha-tocopherol",
    "ahcys__L": "S-Adenosyl-L-homocysteine",
    "amet__L": "S-Adenosyl-L-methionine",
    "gthrd": "Reduced glutathione",
    "avite1r": "Alpha-tocopheroxyl radical",
}
df.index = [mets_map[met] for met in df.index]


fig, ax = plt.subplots(1, 1, figsize=(5, 2.5))
sns.despine(fig)

barsize = 0.75
edgecolor = "black"
edgewidth = 1
padding = 2
offset = np.zeros(len(df.index))
df.index = df.index.astype(str)
df = df.loc[df.index[::-1]]
for idx, (label, series) in enumerate(df.items()):
    rects = ax.barh(
        series.index,
        series.values,
        height=barsize,
        left=offset,
        color=colormaps_dict.get(label),
        edgecolor=edgecolor,
        linewidth=edgewidth,
    )
    offset += series.values

ax.xaxis.set_tick_params(labelsize="large")
ax.yaxis.set_tick_params(labelsize="large")

ax.set_xlabel("Number of associated reactions", fontdict={"size": "large"})
ax.set_xticks([0, 30, 60, 90, 120])

fig.tight_layout()
if save_figures:
    fig.savefig(
        results_path / f"Top10_wo_transporter.{imagetype}",
        transparent=transparent,
        dpi=None if imagetype != "png" else dpi,
    )